# LnagCahain trial

In [ ]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_community.llms import Ollama
import pandas as pd



In [ ]:


# Load and preprocess the dataset
def preprocess_dataset(file_path, relevant_columns):
    """
    Preprocess the Quranic dataset by removing unnecessary columns.
    Args:
    - file_path: Path to the CSV file.
    - relevant_columns: List of columns to retain.

    Returns:
    - Processed DataFrame.
    """
    try:
        df = pd.read_csv(file_path)
        return df[relevant_columns]
    except Exception as e:
        print(f"Error in loading or processing dataset: {e}")
        return pd.DataFrame()  # Return an empty DataFrame if error occurs


# Specify relevant columns
relevant_columns = ['surah_name_roman', 'surah_name_en', 'ayah_no_surah', 'ayah_en']
quran_df = preprocess_dataset("TheQuranDataset.csv", relevant_columns)

if quran_df.empty:
    print("The dataset could not be loaded. Please check the file path and content.")
    exit()  # Exit if the dataset is not properly loaded





# Chunk the dataset for LLM processing
def preprocess_and_chunk(df, chunk_size=4000):
    """
    Preprocesses the dataset to create formatted context chunks.
    Args:
    - df: Processed DataFrame.
    - chunk_size: Maximum character size for each chunk.

    Returns:
    - List of context chunks (list of str).
    """
    context_lines = [
        f"Ayah {row['ayah_no_surah']} of Surah {row['surah_name_roman']} ({row['surah_name_en']}): "
        f"'{row['ayah_en']}'"
        for _, row in df.iterrows()
    ]
    combined_context = "\n".join(context_lines)

    # Chunk the combined context
    context_chunks = [
        combined_context[i:i + chunk_size]
        for i in range(0, len(combined_context), chunk_size)
    ]
    return context_chunks


context_chunks = preprocess_and_chunk(quran_df)

if not context_chunks:
    print("No context chunks created. Please check the dataset or preprocessing logic.")
    exit()  # Exit if no chunks are created


# Query definition
query = "Does the Quran mention dogs? Provide references and context."

# Define a prompt template
prompt_template = """
Use the following Quranic data to answer the query and provide references with context.

Context:
{context}

Query: {question}
"""

# Create a LangChain PromptTemplate
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template
)

# Initialize the Ollama LLM
llm = Ollama(model="llama3")

# Create an LLMChain for structured execution
llm_chain = LLMChain(llm=llm, prompt=prompt)

# Execute the query on each chunk
def execute_query_on_chunks(context_chunks, query):
    """
    Executes a query on context chunks using the LLM chain.
    Args:
    - context_chunks: List of text chunks.
    - query: Query string.

    Returns:
    - Combined results from all chunks.
    """
    results = []
    for chunk in context_chunks:
        inputs = {"context": chunk, "question": query}
        try:
            result = llm_chain.run(inputs)
            results.append(result)
        except Exception as e:
            print(f"Error processing chunk: {e}")
    return "\n".join(results)


final_result = execute_query_on_chunks(context_chunks, query)

# Print the final output
print("\nOutput:")
print(final_result)


eVALUATE

In [ ]:


# Keyword search function
def keyword_search(keywords, df):
    """
    Searches for ayahs containing specific keywords in the Quranic dataset.
    Args:
    - keywords: List of keywords to search for.
    - df: DataFrame containing Quranic data.

    Returns:
    - DataFrame with matched ayahs and relevant details.
    """
    matches = df[
        df['ayah_en'].str.contains('|'.join(keywords), case=False, na=False)
    ]
    return matches


# Keywords related to "dogs"
keywords = ["dog", "dogs", "dogs talk", "speak"]
matched_ayahs = keyword_search(keywords, quran_df)

# Display the results in a clean table format
if not matched_ayahs.empty:
    print("\nMatched Ayahs:")
    print(
        matched_ayahs.to_markdown(index=False)  # Use Markdown for table-like display
    )
else:
    print("\nNo matches found for the given keywords.")


# Build a Basic LangChain Pipeline

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.llms import OpenAI

# Initialize OpenAI model
llm = OpenAI(model_name="text-davinci-003", temperature=0.7)

# Create a prompt template
template = """
You are a helpful assistant. Answer the user's question based on the document provided.

Document: {document}

Question: {question}

Answer:
"""
prompt = PromptTemplate(template=template, input_variables=["document", "question"])

# Build the chain
qa_chain = LLMChain(llm=llm, prompt=prompt)

# Provide a sample document and question
document = "Artificial Intelligence (AI) involves creating systems capable of performing tasks that typically require human intelligence."
question = "What is AI?"

# Run the chain
response = qa_chain.run({"document": document, "question": question})
print("Response:", response)


# Document Retrieval with Embeddings

In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings

# Sample documents
documents = [
    "AI is transforming industries.",
    "AI applications include NLP and robotics.",
    "Machine learning is a subset of AI."
]

# Create embeddings
embedding_model = OpenAIEmbeddings()
vector_store = Chroma.from_texts(documents, embedding=embedding_model)

# Query the vector store
query = "Tell me about AI applications."
results = vector_store.similarity_search(query, k=2)

print("Retrieved Documents:")
for doc in results:
    print(doc.page_content)


# Conversational Memory

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

# Initialize memory
memory = ConversationBufferMemory()

# Create a conversational chain
conversation = ConversationChain(llm=llm, memory=memory)

# Simulate a conversation
response1 = conversation.predict(input="What is AI?")
response2 = conversation.predict(input="Tell me about its applications.")

print("Conversation:")
print(response1)
print(response2)


# Working with Agents

In [ ]:
from langchain.agents import Tool, initialize_agent
from langchain.llms import OpenAI
from langchain.tools import tool

# Define a tool
@tool
def calculator(expression: str) -> str:
    try:
        return str(eval(expression))
    except Exception as e:
        return str(e)

tools = [Tool(name="Calculator", func=calculator, description="Performs calculations.")]

# Initialize an agent
llm = OpenAI(model_name="text-davinci-003")
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

# Query the agent
response = agent.run("What is 12 multiplied by 8?")
print("Agent Response:", response)


# Test Document Retrieval Pipelines with Embeddings

In [ ]:
from langchain.chains import RetrievalQA

# Combine retrieval with QA
retrieval_qa = RetrievalQA.from_chain_type(llm=llm, retriever=vector_store.as_retriever())

# Ask a question
question = "What is AI?"
response = retrieval_qa.run(question)
print("Response:", response)


# Deploy Locally or on the Cloud

In [ ]:
%pip install streamlit


In [ ]:
import streamlit as st
from langchain.chains import ConversationChain
from langchain.llms import OpenAI

# Initialize the conversational chain
llm = OpenAI(model_name="text-davinci-003")
conversation = ConversationChain(llm=llm)

# Streamlit UI
st.title("Conversational AI with LangChain")
user_input = st.text_input("You: ", "")
if user_input:
    response = conversation.predict(input=user_input)
    st.text_area("AI:", response, height=200)


In [ ]:
streamlit run app.py

In [ ]:
from langchain.chains import LLMChain, ConversationalRetrievalChain
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.memory import ConversationBufferMemory
from langchain.agents import initialize_agent, Tool
from langchain.tools import GoogleSearchRun
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings

# Step 1: Prompt Template
template = """You are an intelligent assistant. Here is the user's question:
{question}

Provide a concise, helpful response:
"""
prompt = PromptTemplate(input_variables=["question"], template=template)

# Step 2: Memory for Context
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# Step 3: LLM (Language Model)
llm = OpenAI(model="text-davinci-003", temperature=0.3)

# Step 4: Vector Database for Document Retrieval
# Create a small set of documents
documents = [
    "LangChain is a framework for building applications with language models.",
    "OpenAI's GPT-3 is a widely used language model for text generation.",
    "LangChain enables retrieval-augmented generation (RAG) with embeddings.",
]

# Generate embeddings and create a vectorstore
embedding = OpenAIEmbeddings()
vectorstore = Chroma.from_texts(documents, embedding)
retriever = vectorstore.as_retriever()

# Step 5: Define Tools
search_tool = Tool(
    name="GoogleSearch",
    func=GoogleSearchRun().run,
    description="Search Google for information on any topic."
)

# Step 6: Conversational Retrieval Chain
retrieval_chain = ConversationalRetrievalChain(
    retriever=retriever,
    memory=memory,
    llm=llm
)

# Step 7: Create an Agent with Tools
tools = [search_tool]
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", memory=memory)

# Step 8: LLM Chain for Prompt Template
llm_chain = LLMChain(llm=llm, prompt=prompt)

# Step 9: Run the Workflow
# 1. Use the conversational retrieval chain to retrieve knowledge
query = "What is LangChain?"
retrieval_response = retrieval_chain.run(query)
print(f"[Retrieved Knowledge]: {retrieval_response}")

# 2. Use the LLM chain with the prompt template
question = "What is the difference between GPT-3 and LangChain?"
formatted_response = llm_chain.run({"question": question})
print(f"[LLM Response with Prompt]: {formatted_response}")

# 3. Use the agent with tools to get external information
agent_query = "Who is the CEO of OpenAI?"
agent_response = agent.run(agent_query)
print(f"[Agent Response]: {agent_response}")

# 4. Demonstrate memory by asking a follow-up question
follow_up = "What did I just ask you about OpenAI?"
follow_up_response = agent.run(follow_up)
print(f"[Follow-Up with Memory]: {follow_up_response}")


In [ ]:
from langchain.chains import ConversationalRetrievalChain, LLMChain
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.memory import ConversationBufferMemory
from langchain.agents import initialize_agent, Tool
from langchain.tools import GoogleSearchRun
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings

class LangChainChatbot:
    def __init__(self):
        # Step 1: Initialize LLM
        self.llm = OpenAI(model="text-davinci-003", temperature=0.7)

        # Step 2: Initialize Memory
        self.memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

        # Step 3: Initialize Prompt Template
        self.prompt_template = PromptTemplate(
            input_variables=["chat_history", "user_input"],
            template=(
                "You are a helpful chatbot. Here's the conversation so far:\n"
                "{chat_history}\n\n"
                "User: {user_input}\nChatbot:"
            )
        )
        self.llm_chain = LLMChain(llm=self.llm, prompt=self.prompt_template)

        # Step 4: Initialize Document Retrieval (RAG)
        documents = [
            "LangChain is a framework for building applications with language models.",
            "OpenAI's GPT-3 is a popular model for text generation.",
            "LangChain supports tools like embeddings, memory, and agents for reasoning.",
        ]
        embedding = OpenAIEmbeddings()
        self.vectorstore = Chroma.from_texts(documents, embedding)
        self.retriever = self.vectorstore.as_retriever()

        # Step 5: Initialize Tools
        search_tool = Tool(
            name="GoogleSearch",
            func=GoogleSearchRun().run,
            description="Search Google for information on any topic."
        )
        self.tools = [search_tool]

        # Step 6: Initialize Agent
        self.agent = initialize_agent(
            tools=self.tools,
            llm=self.llm,
            agent="zero-shot-react-description",
            memory=self.memory
        )

        # Step 7: Initialize Conversational Retrieval Chain
        self.retrieval_chain = ConversationalRetrievalChain(
            retriever=self.retriever,
            memory=self.memory,
            llm=self.llm
        )

    def chat(self, user_input):
        """
        Handle a single user input and decide whether to use retrieval, tools, or chains.
        """
        try:
            # If the user input is a retrieval-based query
            if "retrieve" in user_input.lower():
                response = self.retrieval_chain.run(user_input)
                return f"[Retrieved Info]: {response}"

            # If the user input requires tool usage (e.g., Google Search)
            elif "search" in user_input.lower():
                response = self.agent.run(user_input)
                return f"[Search Result]: {response}"

            # Default response using prompt templating and memory
            else:
                response = self.llm_chain.run({
                    "chat_history": self.memory.load_memory_variables({}).get("chat_history", ""),
                    "user_input": user_input
                })
                return response
        except Exception as e:
            return f"An error occurred: {e}"

# Main Chatbot Loop
if __name__ == "__main__":
    chatbot = LangChainChatbot()
    print("Chatbot is ready! Type 'exit' to quit.\n")

    while True:
        user_input = input("You: ")
        if user_input.lower() == "exit":
            print("Chatbot: Goodbye!")
            break

        response = chatbot.chat(user_input)
        print(f"Chatbot: {response}")
